# 0. Import Lib

In [1]:
import networkx as nx
import numpy as np
from sthype import SpatialGraph
from sthype.graph_functions import spatial_temporal_graph_from_spatial_graphs
from sthype.plot import plot_spatial_temporal_graph, plot_spatial_temporal_graph_node, plot_spatial_graph, plot_spatial_temporal_graph_hyperedge
import matplotlib.pyplot as plt
import os
import pickle

%load_ext autoreload
%autoreload 2

# 1. Import Graph

In [2]:
graphs = []

str_directory = "/home/ipausers/hamzaoui/Documents/Data/Graphs/482_20230908"
directory = os.fsencode(str_directory)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    file_path = os.path.join(str_directory, filename)
    if filename.endswith('pickle'):
        graphs.append((filename, pickle.load(open(file_path, 'rb'))))

graphs.sort()
graphs = [graph for _, graph in graphs]
graphs

# 2. Set up stg

In [3]:
spatial_graphs = [SpatialGraph(graph) for graph in graphs]
spatial_graph = spatial_graphs[-1]
sg = spatial_graph

In [4]:
spatial_temporal_graph = spatial_temporal_graph_from_spatial_graphs(spatial_graphs, np.arange(len(spatial_graphs)), verbose=1)
stg = spatial_temporal_graph

Segmentation
Edge Activation
Comparing with graph 20
Comparing with graph 19
Comparing with graph 18
Comparing with graph 17
Comparing with graph 16
Comparing with graph 15
Comparing with graph 14
Comparing with graph 13
Comparing with graph 12
Comparing with graph 11
Comparing with graph 10
Comparing with graph 9
Comparing with graph 8
Comparing with graph 7
Comparing with graph 6
Comparing with graph 5
Comparing with graph 4
Comparing with graph 3
Comparing with graph 2
Comparing with graph 1
Comparing with graph 0


# 3. Analysis

In [ ]:
g_t = []
for i in range(21):
    g_t.append(stg.get_graph_at(i))
widths_t = []

hyperedge_e = {}
for u, v, ed in g_t[-1].edges(data=True):
    hyperedge_e.setdefault(ed["hyperedge"], []).append((u, v, ed['activation'], LineString(ed['pixels']).length))

In [ ]:
tips = []
tips_light = []
growth_l = []
growth_s = []
growth_v = []
for edges in hyperedge_e.values():
    for i in range(1, 20):
        widths = [g_t[i+1][u][v]['attributes'].get('width', [np.nan]) for u, v, a, l in edges if a==i]
        widths = [w for ws in widths for w in ws]
        width = np.median(widths)
        if np.isnan(width) or width<=2:
            continue
        lengths = [l for u, v, a, l in edges if a==i+1]
        ss = [np.pi*l*np.median(g_t[a][u][v]['attributes'].get('width', np.nan)) for u, v, a, l in edges if a==i+1]
        vs = [np.pi*l*np.median(g_t[a][u][v]['attributes'].get('width', np.nan))*np.median(g_t[a][u][v]['attributes'].get('width', np.nan))/4 for u, v, a, l in edges if a==i+1]
        length = np.sum(lengths)
        if length==0:
            continue
        tips.append(width)
        growth_l.append(length)
        if np.isnan(ss).any():
            continue
        tips_light.append(width)
        s = np.sum(ss)
        v = np.sum(vs)
        growth_s.append(s)
        growth_v.append(v)